In [68]:
from time import time
import os
import collections

In [217]:
def def_onlyCode(allLines):
    copyline = allLines.copy()
    newLines=[]
    lines=iter(allLines)
    citer=0
    withoutSpace = lambda x : x.replace(' ','')
    blankLine = lambda x : x=='\n' or x==''
    commWildcard = lambda x:x[0]=='#'
    commOneline = lambda x:(initComment(x) and endComment(x[3:]))
    initComment = lambda x:(x[:3]=='"""')
    endComment = lambda x:(x[-4:-1]=='"""')
    for line in lines:
        citer=citer+1
        newLine = withoutSpace(line)
        if blankLine(newLine):
            continue
        if commWildcard(newLine):
            continue
        if commOneline(newLine):
            continue
        if initComment(newLine):
            line = next(lines)
            for i in range(citer,len(copyline)):
                newLine = withoutSpace(line)
                endLine = citer+1==len(copyline)
                line = next(lines)
                if endComment(newLine) or endLine:
                    break
                else:
                    citer=citer+1
        newLines.append(line)   
    return newLines

In [218]:
def depFor(archivo):
    newLineas=archivo.copy()
    lineas=iter(archivo)
    citer=0
    withoutSpace = lambda x : x.replace(' ','')
    initFor = lambda x:(x[:3]=='for')
    endFor = lambda x:(x[-2:-1]==':')
    countSpace = lambda x:(x.index(x.replace(' ','')[0]))
    arrMax=[0]
    arr=[]
    for linea in lineas:
        citer=citer+1
        wordFor='for'
        wordEnd=':'
        space = countSpace(linea)
        patronFor=((initFor(withoutSpace(linea))) and (endFor(withoutSpace(linea))))
        if patronFor:
            newLines=[]
            for i in range(citer,len(newLineas)):
                endLine = citer+1==len(newLineas)
                forSpace = False
                if not endLine:
                    nextspaceLine = countSpace(newLineas[citer])
                    forSpace = nextspaceLine<=space
                else:
                    try:
                        newLines.append(next(lineas))
                    except:
                        pass
                if forSpace or endLine:
                    value,arr=depFor(newLines)
                    arrMax.append(value+1)
                    break
                else:
                    try:
                        newLines.append(next(lineas))
                        citer=citer+1
                    except:
                        pass
    return(max(arrMax),arrMax)

In [219]:
def librerias(lines):
    setlibrerias=set()
    wordImport= 'import'
    wordFrom= 'from'
    notElement={'import', 'from', ''}
    for line in lines:
        inImport=(wordImport in line)
        inFrom=(wordFrom in line)
        if inFrom and inImport:
            arrlibr = line[:line.index('import')].split(' ')
            setlibrerias = setlibrerias|set(arrlibr)
            continue
        if inImport:
            arrlibr =  line[:-1].split(' ')
            setlibrerias = setlibrerias|set(arrlibr)
            continue
    return setlibrerias-notElement

In [220]:
def countDef(lines):
    wordDef='def'
    wordEnd=':'
    arrArg=[]
    for line in lines:
        if wordDef in line and wordEnd in line:
            argini=line.find("(")+1
            argend=line.find(")")
            arg=line[argini:argend].split(',')
            arrArg.append(len(arg))
    return arrArg

In [221]:
def duplicateLine(arrlines):
    newGroup=[]
    withoutSpace = lambda x : x.replace(' ','')
    lines=list(map(withoutSpace,arrlines))
    unionLine=lambda x : x.replace('[','').replace(',','').replace(']','')
    for i,line in enumerate(lines):
        newGroup.append(str(unionLine(str(lines[i:i+4]))))
    repetidos=collections.Counter(newGroup)
    duplicado = [1 for key in repetidos if repetidos[key]>=2]
    return (sum(duplicado)+3*bool(duplicado))

In [222]:
def returnFilespy(carpeta):
    newArray=[]
    for archivo in os.listdir(carpeta):
        if os.path.isdir(os.path.join(carpeta,archivo)):
            newArray = newArray+returnFilespy(os.path.join(carpeta,archivo))
        else:
            newArray.append(os.path.join(carpeta,archivo))
    filesPy = [filePy for filePy in newArray if filePy[-3:]=='.py' ]
    return filesPy

In [226]:
ruoteFlies = '.\\Almacen_repositorios\\data_hacks-master'
filesPy = returnFilespy(ruoteFlies)
filesPy

['.\\Almacen_repositorios\\data_hacks-master\\data_hacks\\bar_chart.py',
 '.\\Almacen_repositorios\\data_hacks-master\\data_hacks\\histogram.py',
 '.\\Almacen_repositorios\\data_hacks-master\\data_hacks\\ninety_five_percent.py',
 '.\\Almacen_repositorios\\data_hacks-master\\data_hacks\\run_for.py',
 '.\\Almacen_repositorios\\data_hacks-master\\data_hacks\\sample.py',
 '.\\Almacen_repositorios\\data_hacks-master\\setup.py']

In [227]:
start_time = time()
for filePy in filesPy:
    print('********************************************************')
    print(filePy)
    file = open(filePy, 'r')
    lines = file.readlines()
    linesCode = def_onlyCode(lines)
    print(len(linesCode))
    countFor = depFor(linesCode)
    print(countFor[1][1:])
    libreriasSet = librerias(linesCode)
    print(libreriasSet)
    countArgdef = countDef(linesCode)
    print(countArgdef)
    isDupplicate = duplicateLine(linesCode)
    print(isDupplicate)
    file.close()
    elapsed_time = time() - start_time
    print(elapsed_time)

********************************************************
.\Almacen_repositorios\data_hacks-master\data_hacks\bar_chart.py
80
[1, 1, 1]
{'collections', 'decimal', 'optparse', 'sys', 'math'}
[1, 2]
0
0.010959386825561523
********************************************************
.\Almacen_repositorios\data_hacks-master\data_hacks\histogram.py
216
[1, 1, 1, 1, 1, 1, 2, 1]
{'collections', 'decimal', 'optparse', 'sys', 'math', 'logging'}
[1, 3, 1, 1, 1, 1, 3, 2, 1, 2, 2, 2]
0
0.08575844764709473
********************************************************
.\Almacen_repositorios\data_hacks-master\data_hacks\ninety_five_percent.py
32
[1, 1]
{'os', 'sys', 'decimal'}
[1, 2]
0
0.13164114952087402
********************************************************
.\Almacen_repositorios\data_hacks-master\data_hacks\run_for.py
37
[1]
{'sys', 'os', 'time'}
[1, 1]
0
0.1944713592529297
********************************************************
.\Almacen_repositorios\data_hacks-master\data_hacks\sample.py
38
[1]
{'rando